In [25]:
import os
import sys
import time

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision.models import resnet50
from tqdm import tqdm

sys.path.append("../src")

In [26]:
import utils.config as cfg
from models.detection_model import BboxDetectionModel
from datasets.landmark_dataset import LandmarkDataset
from utils.models_utils import ModelUtils, TrainTransforms

Mean and std values are specified for ImageNet deviation. *temp solution*, needs to be determined for DeepFashion dataset

In [27]:
#MEAN, STD and RESIZE_SIZE declaration
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
RESIZE_SIZE =(400,300)
BBOX_WEIGHT = 1.0
LABEL_WEIGHT = 1.0

In [28]:
INITIAL_LR = 1e-4
NUM_EPOCHS = 20
BATCH_SIZE = 16

Memory allocation

In [29]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIN_MEMORY = True if DEVICE == "cuda" else False # to speed up loading data on CPU to training it on GPU
# see https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723

Loading dataset

In [30]:

tf_list = TrainTransforms(mean=MEAN,std=STD,resize_size=RESIZE_SIZE)


In [31]:
dataset = LandmarkDataset(root="../"+cfg.LANDMARK_DATASET_PATH, runing_mode="train",device="cpu",transforms_list=tf_list )
train_dataloader = DataLoader(dataset=dataset,batch_size=BATCH_SIZE,num_workers=os.cpu_count())

In [32]:
# resnet = resnet50()
resnet_utils = ModelUtils()
resnet_model =resnet_utils.build_model(model = resnet50,pretrained=True,gradation=False)
#creating object dedector

#TODO Bbox detection Model class needs to be refilled
detector_model =  BboxDetectionModel(resnet_model,3)
if torch.cuda.is_available():
    detector_model.cuda()

[INFO]loading model with pretrained weigts.
[INFO] Freezing all hiden layers.


/home/filippatyk/anaconda3/envs/clothes/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/filippatyk/anaconda3/envs/clothes/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [33]:
#loss funcfion declaration
classificator_loss_func  = nn.CrossEntropyLoss()
bbox_loss_func = nn.MSELoss()
#optimizer initialization
optimizer = torch.optim.Adam(params = detector_model.parameters(),lr=INITIAL_LR)

In [34]:
# initialize a dictionary to store training history
H = {"total_train_loss": [], "total_val_loss": [], "train_class_acc": [],
	 "val_class_acc": []}



In [41]:
print(f"{cfg.TERMINAL_INFO} start training loop...")
start_time = time.time()

for epoch in tqdm(range(NUM_EPOCHS)):
    detector_model.train()
    
    # initialization losses
    total_train_loss = 0
    total_valid_loss = 0

    # initialization predictions num
    train_correct = 0
    valid_correct = 0
    i = 0
    for (image,label,bbox) in train_dataloader:
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        bbox  = bbox.to(DEVICE)

        predictions = detector_model(image)
        bbox_loss = bbox_loss_func(predictions[0],bbox)
        classificator_loss = classificator_loss_func(predictions[1],label)
        total_loss =  bbox_loss.float() +  classificator_loss.float()
        print(type(total_loss))
        print(total_loss.dtype)
        print(bbox_loss.dtype)
        print(classificator_loss.dtype)


        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_train_loss += total_loss
        train_correct += (predictions[1].argmax(1) == label).type(torch.float).sum().item()
        

        #TODO valid loop
        

        

[INFO] start training loop...


  0%|          | 0/20 [00:00<?, ?it/s]

<class 'torch.Tensor'>
torch.float32
torch.float32
torch.float32


  0%|          | 0/20 [00:17<?, ?it/s]


RuntimeError: Found dtype Long but expected Float